# Regular Representation of a Group

## References

* Georgi, Howard (2018), <i>"Lie algebras in particle physics: from isospin to unified theories"</i>, CRC Press, [Open Access](https://www.taylorfrancis.com/books/oa-mono/10.1201/9780429499210/lie-algebras-particle-physics-howard-georgi?context=ubx&refId=1530fefc-3778-48ae-99ec-cba2935af2fb)
* Huang, Jiaqi (2012), <i>"Lie Groups and their applications to Particle Physics: A Tutorial for Undergraduate Physics Majors"</i>, [arXiv:2012.00834v1](https://arxiv.org/abs/2012.00834)

## Introduction

The following definition was adapted from the definition in [Georgi, 2018].

A <i>representation</i> of a group, $G = \langle A, \circ \rangle$, is a mapping, $V$, of the elements of $G$ onto a set of linear operators with the following properties:
* $V(e) = \hat{1}$, where $e$ is the group's identity element and $\hat{1}$ is the identity operator in the space on which the linear operators act
* $V(a_i) \cdot V(a_j) = V(a_i \circ a_j)$. That is, the group multiplication law (denoted by $\circ$) is mapped onto the natural multiplication in the linear space on which the linear operators act (denoted by $\cdot$).

The <i>regular representation</i> of a group is a mapping of each group element to an $nxn$ matrix. In this case, $\hat{1}$ is the $nxn$ identity matrix.

For example, the regular representation of the cyclic group, $Z_4$, with elements, $A = \left\{ R_0, R_{90}, R_{180}, R_{270} \right\}$, consists of the mapping shown below, as a Python dictionary.

In [1]:
import numpy as np  # Use NumPy to represent matrices & vectors

In [2]:
z4_reg_rep = {
    'R0': np.array([[1, 0, 0, 0],
                    [0, 1, 0, 0],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]]),
    'R90': np.array([[0, 0, 0, 1],
                     [1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0]]),
    'R180': np.array([[0, 0, 1, 0],
                      [0, 0, 0, 1],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0]]),
    'R270': np.array([[0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1],
                      [1, 0, 0, 0]])
}

## Theory

The following description of the algorithm for computing a regular representation was adapted from [Huang, 2012].

Let $G = \langle A, \circ \rangle$, be a group, where $A = \{a_0, a_1, \dots , a_{n - 1}\}$ is the set of the group's elements, and $\circ$ is its binary operator.

Also, let $B = \{\hat{b}_0, \hat{b}_1, \dots , \hat{b}_{n-1} \}$ be a set of $nx1$ orthogonal unit vectors:

$\hat{b}_0 = \begin{bmatrix}
1 \\
0 \\
0 \\
\vdots \\
0 \end{bmatrix},
\hat{b}_1 = \begin{bmatrix}
0 \\
1 \\
0 \\
\vdots \\
0 \end{bmatrix},
\dots,
\hat{b}_{n-1} = \begin{bmatrix}
0 \\
0 \\
0 \\
\vdots \\
1 \end{bmatrix}$

Define the a bijection between $A$ and $B$ as follows: $V(a_i) = \hat{b}_i$ for $i = 0, \dots , n - 1$.

Let $\cdot$ denote matrix-vector multiplication, and define the $nxn$ matrix,

$C_k = (c^k_{ij})_{i,j=0,\dots,n-1}$

where $c^k_{ij} = \hat{b}_i^T \cdot V(a_k \circ V^{-1}(\hat{b}_j))$



Then $M = \{C_0, C_1, \dots , C_{n - 1}\}$ is the <b>regular representation</b> of the group $G$, where the mapping between group elements and operators is $a_i \leftrightarrow C_i$ for $i = 0, \dots , n - 1$.

The purpose of the remainder of this notebook is to develop an implementation, within the module <b>finite_algebras</b>, that produces the mapping, $a_i \leftrightarrow C_i$, from a given group, $G$.

## A Small Test Group

In [3]:
import finite_algebras as alg

In [4]:
grp = alg.make_finite_algebra(
    'Z4',
    'Cyclic group of order 4',
    ['R0', 'R90', 'R180', 'R270'],
    [[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]
)

grp.about()  # Displays information about the group


** Group **
Name: Z4
Instance ID: 4758106320
Description: Cyclic group of order 4
Order: 4
Identity: R0
Commutative? Yes
Cyclic?: Yes
  Generators: ['R270', 'R90']
Elements:
   Index   Name   Inverse  Order
      0      R0      R0       1
      1     R90    R270       4
      2    R180    R180       2
      3    R270     R90       4
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]


In [5]:
a = grp.elements
n = grp.order

## The Bijection between Group Elements and Vectors

First create a dictionary that maps each of the group's elements to an n-dimensional orthogonal basis vector.

In [6]:
id = np.eye(n, dtype=int)  # The nxn identity matrix
b = [id[:,[i]] for i in range(n)]  # A list of the columns of the identity matrix
mapping = dict(zip(a, b))
mapping

{'R0': array([[1],
        [0],
        [0],
        [0]]),
 'R90': array([[0],
        [1],
        [0],
        [0]]),
 'R180': array([[0],
        [0],
        [1],
        [0]]),
 'R270': array([[0],
        [0],
        [0],
        [1]])}

The dictionary, above, maps group elements to "vectors" (NumPy arrays). To complete the bijection, we need another dictionary that goes in the reverse direction, "vectors" to group elements. However, dictionary keys must be immutable, and NumPy arrays are mutable, so the code, below, transforms NumPy arrays to tuples, which <i>are</i> immutable.

In [7]:
def to_tuple(vec):
    '''Turns a column vector into a tuple, for use as a dictionary key.'''
    return tuple(map(lambda x: x[0], list(vec)))

inv_mapping = {to_tuple(val): key for key, val in mapping.items()}
inv_mapping

{(1, 0, 0, 0): 'R0',
 (0, 1, 0, 0): 'R90',
 (0, 0, 1, 0): 'R180',
 (0, 0, 0, 1): 'R270'}

Now define the bijection. That is, $V$, and its inverse, $V^{-1}$.

In [8]:
def V(elem):
    '''Given a group element name, return the corresponding vector.'''
    return mapping[elem]

def Vinv(vec):
    '''Given a vector, return the corresponding element.'''
    return inv_mapping[to_tuple(vec)]

# Test/Example:
elem = a[1]
vec = V(elem)
elem2 = Vinv(vec)
print(elem)
print(vec)
print(elem2)

R90
[[0]
 [1]
 [0]
 [0]]
R90


## The Regular Representation Matrices

Now, the regular represention matrices can be derived using the formula presented in the Theory section, above,

$C_k = (c^k_{ij})_{i,j=0,\dots,n-1}$ where $c^k_{ij} = \hat{b}_i^T \cdot V(a_k \circ V^{-1}(\hat{b}_j))$

In [9]:
reg_rep = dict()

for k in range(n):
    c_k = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            c_k[i][j] = np.dot(b[i].transpose(), V(grp.op(a[k], Vinv(b[j]))))
    reg_rep[a[k]] = c_k

reg_rep

{'R0': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'R90': array([[0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]),
 'R180': array([[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]]),
 'R270': array([[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]])}

### Check Answer

Test to see if each matrix, computed for <b>reg_rep</b>, equals its counterpart in <b>z4_reg_rep</b>, defined in the Introduction section, above.

In [10]:
all([np.array_equal(reg_rep[elem], z4_reg_rep[elem]) for elem in grp.elements])

True

## Putting it All Together

Here is the desired implementation.

In [11]:
def regular_representation(grp):
    '''Given a group, return a dictionary that maps each group element to its corresponding
    regular representation.'''
    
    a = grp.elements
    n = grp.order
    
    # Create a list of n nx1 orthogonal unit vectors
    id = np.eye(n, dtype=int)  # The nxn identity matrix
    b = [id[:,[i]] for i in range(n)]  # A list of the columns of the identity matrix
    
    # map group elements to vectors
    mapping = dict(zip(a, b))

    def V(elem):
        return mapping[elem]

    # Turn a column vector into a tuple, for use as a dict key
    def to_tuple(vec):
        return tuple(map(lambda x: x[0], list(vec)))
    
    # map vectors to group elements
    inv_mapping = {to_tuple(val): key for key, val in mapping.items()}
    
    def Vinv(vec):
        return inv_mapping[to_tuple(vec)]

    # Derive the n nxn matrices of the regular representation
    reg_rep = dict()
    for k in range(n):
        c_k = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                c_k[i][j] = np.dot(b[i].transpose(), V(grp.op(a[k], Vinv(b[j]))))
        reg_rep[a[k]] = c_k

    return reg_rep

In [12]:
reg_rep = regular_representation(grp)
reg_rep

{'R0': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'R90': array([[0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]),
 'R180': array([[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]]),
 'R270': array([[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]])}

In [13]:
def array_2d_to_tuple(arr):
    '''Turn a 2D nd.array into a tuple of tuples, for use as a dictionary key'''
    return tuple(map(lambda x: tuple(x), np.ndarray.tolist(arr)))

# Small test:
mapping_2d = array_2d_to_tuple(reg_rep['R90'])
mapping_2d

((0.0, 0.0, 0.0, 1.0),
 (1.0, 0.0, 0.0, 0.0),
 (0.0, 1.0, 0.0, 0.0),
 (0.0, 0.0, 1.0, 0.0))

In [14]:
inv_mapping_2d = {array_2d_to_tuple(arr): key for key, arr in reg_rep.items()}
inv_mapping_2d

{((1.0, 0.0, 0.0, 0.0),
  (0.0, 1.0, 0.0, 0.0),
  (0.0, 0.0, 1.0, 0.0),
  (0.0, 0.0, 0.0, 1.0)): 'R0',
 ((0.0, 0.0, 0.0, 1.0),
  (1.0, 0.0, 0.0, 0.0),
  (0.0, 1.0, 0.0, 0.0),
  (0.0, 0.0, 1.0, 0.0)): 'R90',
 ((0.0, 0.0, 1.0, 0.0),
  (0.0, 0.0, 0.0, 1.0),
  (1.0, 0.0, 0.0, 0.0),
  (0.0, 1.0, 0.0, 0.0)): 'R180',
 ((0.0, 1.0, 0.0, 0.0),
  (0.0, 0.0, 1.0, 0.0),
  (0.0, 0.0, 0.0, 1.0),
  (1.0, 0.0, 0.0, 0.0)): 'R270'}